In [15]:
# @title 1. Instalação (InsightFace + Ferramentas de Texto)
!pip install insightface onnxruntime opencv-python matplotlib thefuzz -q

In [ ]:
# @title 2. Conectar ao Google Drive e Carregar Banco de Dados
from google.colab import drive
import pickle
import os
import numpy as np
from thefuzz import process # Para comparar textos parecidos

# 1. Montar o Drive
drive.mount('/content/drive')

# 2. Configurar caminho do Banco de Dados
pasta_chamadinha = '/content/drive/MyDrive/OUTROS/Projetos/Chamadinha'
caminho_db = os.path.join(pasta_chamadinha, 'banco_rostos.pkl')

# Criar pasta se não existir
if not os.path.exists(pasta_chamadinha):
    os.makedirs(pasta_chamadinha)
    print(f"Pasta criada: {pasta_chamadinha}")

# 3. Função para Carregar/Salvar
def carregar_banco():
    if os.path.exists(caminho_db):
        with open(caminho_db, 'rb') as f:
            return pickle.load(f)
    return [] # Retorna lista vazia se não existir banco ainda

def salvar_banco(dados):
    with open(caminho_db, 'wb') as f:
        pickle.dump(dados, f)
    print("Banco de dados atualizado e salvo no Drive!")

# Carrega o banco atual na memória
banco_conhecido = carregar_banco()
print(f"Banco carregado. Alunos conhecidos até agora: {len(banco_conhecido)}")

In [ ]:
# @title 3. Funções de IA (Reconhecimento Facial e Correção de Nomes)

# Função para comparar rosto novo com o banco
def buscar_melhor_match(novo_embedding, banco_dados, threshold=25.0):
    """
    Compara o rosto novo com todos do banco.
    Retorna: (Nome Encontrado, Similaridade) ou (None, 0)
    Quanto MENOR a distância, MAIOR a semelhança (para Euclidiana).
    Aqui converteremos para um 'score' aproximado para facilitar.
    """
    melhor_nome = None
    menor_distancia = float('inf')

    for pessoa in banco_dados:
        db_embedding = pessoa['embedding']
        # Cálculo de Distância Euclidiana (padrão para verificar similaridade)
        distancia = np.linalg.norm(novo_embedding - db_embedding)

        if distancia < menor_distancia:
            menor_distancia = distancia
            melhor_nome = pessoa['nome']

    # Se a distância for muito grande, não é a mesma pessoa
    # Ajuste esse 1.2 conforme testes. InsightFace varia entre 0.8 e 1.4 geralmente
    if menor_distancia < 1.1:
        # Transforma distancia em "certeza" (apenas visual)
        score = max(0, (1.5 - menor_distancia) * 100)
        return melhor_nome, score

    return None, 0.0

# Função para sugerir correção de nome (ex: "Rafa" -> "Rafael")
def verificar_nome_parecido(nome_digitado, banco_dados):
    lista_nomes = list(set([p['nome'] for p in banco_dados])) # Lista única de nomes
    if not lista_nomes:
        return None

    # Busca o nome mais parecido no banco
    melhor_match, score = process.extractOne(nome_digitado, lista_nomes)

    # Se for muito parecido (>85%), sugere
    if score > 85 and melhor_match != nome_digitado:
        return melhor_match
    return None

In [ ]:
# @title 4. Carregar Foto e Detectar Rostos
import cv2
import insightface
from insightface.app import FaceAnalysis
from google.colab import files
import matplotlib.pyplot as plt

# 1. Upload
print("Faça o upload da FOTO DA CHAMADA:")
uploaded = files.upload()
if not uploaded:
    raise ValueError("Nenhuma imagem enviada.")

nome_arquivo_img = next(iter(uploaded))
imagem = cv2.imread(nome_arquivo_img)

# 2. Inicializar IA
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

# 3. Detectar
rostos_detectados = app.get(imagem)
print(f"\nDetectamos {len(rostos_detectados)} alunos na foto.")
print("Rode a próxima célula para iniciar a chamada interativa.")

Faça o upload da FOTO DA CHAMADA:


In [ ]:
# @title 5. Realizar a Chamada (Interativo)
from IPython.display import clear_output, display, Image
import matplotlib.pyplot as plt # Ensure plt is imported here for clarity as it's used within the cell

# Variáveis da Chamada Atual
chamada_hoje = []
novos_registros_para_banco = []

img_display = imagem.copy()
h_img, w_img, _ = imagem.shape

print("INICIANDO CHAMADA... (Responda nas caixas abaixo)")

for i, rosto in enumerate(rostos_detectados):
    bbox = rosto.bbox.astype(int)
    x1, y1, x2, y2 = bbox
    # Margem de segurança para o recorte
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w_img, x2), min(h_img, y2)

    rosto_crop = imagem[y1:y2, x1:x2]

    # --- LÓGICA DE IDENTIFICAÇÃO ---
    nome_sugerido, confianca = buscar_melhor_match(rosto.embedding, banco_conhecido)

    nome_final = ""

    # Loop de validação para garantir resposta correta
    confirmado = False
    while not confirmado:
        clear_output(wait=True)
        # Mostrar o rosto
        plt.figure(figsize=(2, 2))
        plt.imshow(cv2.cvtColor(rosto_crop, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Aluno {i+1}/{len(rostos_detectados)}")
        plt.show()

        # Adiciona uma instrução clara para o usuário sobre onde encontrar o input
        print("\n----------------------------------------------------------------")
        print("Por favor, digite sua resposta na caixa de texto abaixo da imagem.")
        print("----------------------------------------------------------------\n")

        # CASO 1: IA achou alguém parecido
        if nome_sugerido:
            print(f"A IA acha que é: {nome_sugerido} (Confiança: {confianca:.1f}%)")
            resp = input(f"Este é o(a) {nome_sugerido}? [s/n]: ").lower().strip()

            if resp == 's':
                nome_final = nome_sugerido
                # Reforça o aprendizado salvando este novo embedding também
                novos_registros_para_banco.append({'nome': nome_final, 'embedding': rosto.embedding})
                confirmado = True
            elif resp == 'n':
                # Usuário disse não, vai cair no input manual abaixo
                nome_sugerido = None # Reset nome_sugerido so it goes to manual input
            else:
                print("Resposta inválida. Digite 's' para sim ou 'n' para não.")
                # Loop will re-run as 'confirmado' is still False
                continue # Explicitly continue to restart the loop for invalid input

        # CASO 2: IA não sabe ou Usuário negou a sugestão
        if not nome_sugerido:
            entrada_nome = input("Não identifiquei. Qual o nome do aluno? ").strip()

            if not entrada_nome:
                entrada_nome = "Desconhecido"
                confirmado = True
            else:
                # Verifica typos (ex: Usuário digita 'Gabirel', banco tem 'Gabriel')
                sugestao_typo = verificar_nome_parecido(entrada_nome, banco_conhecido)

                if sugestao_typo:
                    resp_typo = input(f"Encontrei '{sugestao_typo}' no banco. Você quis dizer isso? [s/n]: ").lower()
                    if resp_typo == 's':
                        nome_final = sugestao_typo
                    else:
                        nome_final = entrada_nome # É um nome novo mesmo parecendo outro
                else:
                    nome_final = entrada_nome # Nome totalmente novo

                # Salva no banco (pois é um nome novo ou uma nova foto de alguém)
                novos_registros_para_banco.append({'nome': nome_final, 'embedding': rosto.embedding})
                confirmado = True

    # Adiciona à lista de presença de hoje
    chamada_hoje.append(nome_final)
    print(f"Registrado: {nome_final}")

# --- PÓS CHAMADA ---
clear_output(wait=True)
print("Chamada finalizada!")

# Atualizar o Banco de Dados com os novos aprendizados
if novos_registros_para_banco:
    banco_conhecido.extend(novos_registros_para_banco)
    salvar_banco(banco_conhecido)
    print(f"{len(novos_registros_para_banco)} novas referências faciais salvas no banco.")

In [ ]:
# @title 6. Relatório da Chamada
from datetime import datetime
from collections import Counter

# Dados do relatório
data_hoje = datetime.now().strftime("%d/%m/%Y")
hora_hoje = datetime.now().strftime("%H:%M")
total_presentes = len(chamada_hoje)
lista_nomes_limpa = sorted([nome for nome in chamada_hoje if nome != "Desconhecido"])
desconhecidos = chamada_hoje.count("Desconhecido")

# Input opcional da turma
turma_nome = input("Qual o nome da turma (ex: Noite 19h)? ")

print("\n" + "="*40)
print(f"RELATÓRIO DE CHAMADA - {turma_nome}")
print("="*40)
print(f"Data: {data_hoje}")
print(f"Horário do processamento: {hora_hoje}")
print(f"Total de Presentes: {total_presentes}")
if desconhecidos > 0:
    print(f"(Sendo {desconhecidos} não identificados)")
print("-" * 40)
print("ALUNOS PRESENTES:")
for i, nome in enumerate(lista_nomes_limpa):
    print(f"{i+1}. {nome}")
print("="*40)

# (Opcional) Salvar relatório em texto
with open(f"Chamada_{data_hoje.replace('/','-')}_{turma_nome}.txt", "w") as f:
    f.write(f"RELATORIO {data_hoje} - {turma_nome}\n")
    f.write("\n".join(lista_nomes_limpa))
print("Arquivo de texto do relatório salvo.")